<a href="https://colab.research.google.com/github/KTH-EXPECA/examples/blob/main/sdr/setup_sdr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Authentication and Dependencies

Login to Chameleon and download openrc.sh file from [here](https://testbed.expeca.proj.kth.se/project/api_access/openrc/). Upload it here next to this notebook and continue.

In the next cell, we setup the authentication method to be able to use Openstack clients.

In [ ]:
import os, re
from getpass import getpass

with open('/content/demo_project-openrc.sh', 'r') as f:
    script_content = f.read()
    pattern = r'export\s+(\w+)\s*=\s*("[^"]+"|[^"\n]+)'
    matches = re.findall(pattern, script_content)

    for name, value in matches:
        os.environ[name] = value.strip('"')

password = getpass('enter your expeca password:')
os.environ['OS_PASSWORD'] = password

enter your expeca password:··········


Install required packages and dependencies. Ignore the warnings.

In [ ]:
!pip uninstall -q -y moviepy
!pip install -q jedi
!pip install -q git+https://github.com/KTH-EXPECA/python-chi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.9/255.9 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.4/205.4 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.8/487.8 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Import packages

In [ ]:
import json, time
from loguru import logger
import chi.network, chi.container, chi.network
from chi.expeca import reserve, list_reservations, unreserve_byid, get_container_status, wait_until_container_removed, show_reservation_byname, restart_sdr, make_sdr_ni, make_sdr_mango, sdr_tools

# Reserve resources

In the next cell, we reserve the required equipment and resources to form an end to end sdr wifi setup. We reserve 2 SDRs and 1 workers.

In [ ]:
# reserve sdr-07 RJ45
sdr07_rj45_lease = show_reservation_byname("sdr-07-rj45-lease")
if not sdr07_rj45_lease:
    sdr07_rj45_lease = reserve(
        { "type":"network", "name": "sdr-07-rj45", "net_name": "sdr-07-rj45", "segment_id": "113", "duration": { "days":7, "hours":0 } }
    )

# reserve sdr-07 SFP
sdr07_sfp_lease = show_reservation_byname("sdr-07-sfp-lease")
if not sdr07_sfp_lease:
    sdr07_sfp_lease = reserve(
        { "type":"network", "name": "sdr-07-sfp", "net_name": "sdr-07-sfp", "segment_id": "114", "duration": { "days":7, "hours":0 } }
    )

# reserve worker-02
worker02_lease = show_reservation_byname("worker-02-lease")
if not worker02_lease:
    worker02_lease = reserve(
        { "type":"device", "name":"worker-02", "duration": { "days":7, "hours":0 } }
    )
worker02_reservation_id = worker02_lease["reservations"][0]["id"]


leaseslist = list_reservations(brief=True)
print(json.dumps(leaseslist,indent=4))

[
    {
        "name": "sdr-09-lease",
        "id": "0c9e0e6e-d013-425f-a390-8e0ba8f97b4e",
        "reservation_id": "83497084-1f33-4d3c-b19b-ccb24d17207d",
        "status": "ACTIVE",
        "end_date": "2023-09-08T07:43:00.000000"
    },
    {
        "name": "sdr-04-lease",
        "id": "a1b54b97-a4f0-4b7f-be68-e98dae815019",
        "reservation_id": "82cc3ae6-9262-48bb-91e8-0c3b02024566",
        "status": "ACTIVE",
        "end_date": "2023-09-08T07:12:00.000000"
    },
    {
        "name": "worker-10-lease",
        "id": "a504e12a-8564-4b02-9c4c-fb19b46e6d7f",
        "reservation_id": "8504bcba-6b19-4344-a1ba-9c5a3f550831",
        "status": "ACTIVE",
        "end_date": "2023-09-07T16:25:00.000000"
    }
]


# Configure the SDRs

## Change the sdrs' firmwares to Mango

In [ ]:
# change sdr-04 design to ni using worker-02
sdrname = "sdr-07"
sdrnet = chi.network.get_network(sdrname+"-rj45-net")
make_sdr_ni(sdrname,sdrnet['id'],worker02_reservation_id,"ens1f1")

# wait 10 seconds
#time.sleep(10)

# change sdr-09 design to mango using worker-02
#sdrname = "sdr-07"
#sdrnet = chi.network.get_network(sdrname+"-rj45-net")
#make_sdr_mango(sdrname,sdrnet['id'],worker02_reservation_id,"ens1f1")

2023-09-01 08:04:04.639 | SUCCESS  | chi.expeca:make_sdr_mango:299 - created make-sdr-mango container.
2023-09-01 08:04:04.641 | INFO     | chi.expeca:make_sdr_mango:301 - waiting 2 minutes for the sdr-09 to change design.
2023-09-01 08:05:06.410 | SUCCESS  | chi.expeca:make_sdr_mango:315 - SERVICE=change_design env variable is set
running change_design...
You have chosen sdr-09 and design mango
sdr-09 is reachable: {'mango': {'ip': '10.30.1.17', 'port': '22', 'tenant-port': 'te1/0/26', 'up': False}, 'ni': {'ip': '10.30.1.18', 'port': '22', 'tenant-port': 'te4/0/13', 'up': True}}
SSHing to 10.30.1.18, username:root, password: 
running command on 10.30.1.18: 
cp /uboot/mango_bootbin/BOOT.bin /uboot/ ; /sbin/reboot > /dev/null 2>&1 &
command stdout: 

command sdterr: 

Waiting 200 seconds for the new desgin to load
10 seconds to the next poll...
10 seconds to the next poll...
10 seconds to the next poll...
10 seconds to the next poll...
10 seconds to the next poll...
sdr-09 design has be

## Reboot the SDRs

In [ ]:
# reset SDR-07 using worker-02
sdrname = "sdr-07"
sdrnet = chi.network.get_network(sdrname+"-rj45-net")
restart_sdr(sdrname,sdrnet['id'],worker02_reservation_id,"ens1f1")


2023-09-01 10:10:24.442 | SUCCESS  | chi.expeca:restart_sdr:255 - created reboot-sdr container.
2023-09-01 10:10:24.445 | INFO     | chi.expeca:restart_sdr:257 - waiting 2 minutes for the sdr-09 to reboot.
2023-09-01 10:11:15.129 | SUCCESS  | chi.expeca:restart_sdr:267 - SERVICE=reboot env variable is set
running reboot...
You have chosen sdr-09
sdr-09 is reachable: {'mango': {'ip': '10.30.1.17', 'port': '22', 'tenant-port': 'te1/0/26', 'up': True}, 'ni': {'ip': '10.30.1.18', 'port': '22', 'tenant-port': 'te4/0/13'}}
SSHing to 10.30.1.17, username:root, password: root
running command on 10.30.1.17: 
/sbin/reboot > /dev/null 2>&1 &
Waiting 200 seconds for the sdr to load
10 seconds to the next poll...
10 seconds to the next poll...
10 seconds to the next poll...
10 seconds to the next poll...
sdr-09 mango is up again.

2023-09-01 10:11:15.612 | INFO     | chi.expeca:wait_until_container_removed:30 - waiting 30 seconds for reboot-sdr container to be removed
2023-09-01 10:11:20.836 | INFO